In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
from IPython.display import display
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.signal
import os
import pickle

# Set matplotlib parameters for displaying graphs
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = [10, 5]  # Bigger images
plt.rcParams['font.size']= 14

pd.set_option('display.max_columns', None)

In [2]:
cwd = os.getcwd()
home_dir = os.path.dirname(cwd)

In [3]:
filepaths = []
part_paths = []

for subdir, dirs, files in os.walk(home_dir):
    if "part" in subdir:
            part_paths.append(subdir)
    for filename in files:
        filepath = subdir + os.sep + filename
        
        if filepath.endswith(".acq"):
            filepaths.append(filepath)
      
filepaths      

['C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part201\\part201_baseline.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part201\\part201_pilot.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part202\\part202_baseline.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part202\\part202_pilot.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part203\\part203_baseline.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part203\\part203_pilot.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part204\\part204_baseline.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part204\\part204_pilot.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part205\\part205_baseline.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part205\\part205_pilot.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part206\\part206_baseline.acq',
 'C:\\Users\\BIOPACMan\\Documents\\Zhang\\HOME\\data\\part206\\part20

In [4]:
#Process data files into ECG,EDA,RSP signals and SAVE THEM

for file in filepaths:
    # Read in data directly from .acq file and rename dataframe columns
    # ACQ file has pre-named colums so you need to make them easier to deal with later in the script
    # subject numbers are 201-215
    # raw data is barely useful, in units of voltage (RSP), microsiemens (EDA), mV (ECG), and V (TRIG)
    print("PROCESSING...." + file)
    
    data, sampling_rate = nk.read_acqknowledge(file)
    data = data.rename(columns={"RSP, X, RSPEC-R": "RSP", "DTU100 - Trigger View, AMI / HLT - A11": "TRIG",
                                "EDA, X, PPGED-R": "EDA", "ECG, X, RSPEC-R": "ECG"})
    
    # Process ECG, EDA, and RSP over the entire experiment
    # takes the unidimensional psychophys data and derives relevant features for each subject
    file_name = file.replace(".acq","") 
    if not (os.path.exists(file_name+"_ecg_signals.p")):
        ecg_signals, info1 = nk.ecg_process(data["ECG"], sampling_rate=2000)
        pickle.dump( ecg_signals, open( file_name+"_ecg_signals.p", "wb" ) )
        
    if not (os.path.exists(file_name+"_eda_signals.p")):
        eda_signals, info2 = nk.eda_process(data["EDA"], sampling_rate=2000)
        pickle.dump( eda_signals, open( file_name+"_eda_signals.p", "wb" ) )
        
    if not (os.path.exists(file_name+"_rsp_signals.p")):
        rsp_signals, info3 = nk.rsp_process(data["RSP"], sampling_rate=2000)
        pickle.dump( rsp_signals, open( file_name+"_rsp_signals.p", "wb" ) )
      

PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part201\part201_baseline.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part201\part201_pilot.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part202\part202_baseline.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part202\part202_pilot.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part203\part203_baseline.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part203\part203_pilot.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part204\part204_baseline.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part204\part204_pilot.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part205\part205_baseline.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part205\part205_pilot.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhang\HOME\data\part206\part206_baseline.acq
PROCESSING....C:\Users\BIOPACMan\Documents\Zhan

In [ ]:
#Get picklenames
pickle_signals = []
for subdir, dirs, files in os.walk(home_dir):
    for filename in files:
        filepath = subdir + os.sep + filename
        
        if filepath.endswith(".p"):
            pickle_signals.append(filepath)
pickle_signals

In [ ]:
#Code to verify pickling worked!
ecg_signals = []
eda_signals = []
rsp_signals = []
for p in pickle_signals:
    if "ecg" in p:
        ecg = pickle.load(open(p,"rb"))
        ecg_signals.append(ecg)
    if "eda" in p:
        eda = pickle.load(open(p,"rb"))
        eda_signals.append(eda)
    if "rsp" in p:
        rsp = pickle.load(open(p,"rb"))
        rsp_signals.append(rsp)

In [ ]:
filepaths